In [13]:
import base64
import os
import json
import logging
import csv
import time
from google import genai
from google.genai import types

In [7]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(filename)s:%(lineno)d - %(message)s",
    handlers=[
        logging.FileHandler("prompt_generation.log", mode="a")
    ]
)

logger = logging.getLogger("Titty")

In [21]:
client = genai.Client(api_key="your_api_key_here")

In [22]:
generate_content_config = types.GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=genai.types.Schema(
            type = genai.types.Type.OBJECT,
            required = ["prompt"],
            properties = {
                "prompt": genai.types.Schema(
                    type = genai.types.Type.STRING,
                ),
            },
        ),
    )

In [23]:
def generate_prompt(word: str, word_type: str) -> str:
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text=f"""Create a simple, clear, and engaging English textbook illustration prompt for the {word_type} "{word}". 

    Requirements:
    - The illustration should visually represent the meaning of the word.
    - It must NOT contain any text or words in the image.
    - Add that it should follow the style of standard educational textbook illustrations.
    - Provide ONLY ONE well-defined prompt.

    Example Format:
    \"A young girl is sitting on a park bench, reading a book, with a friendly dog lying beside her in a children's English textbook format.\"  

    Now, generate a prompt for the {word_type} "{word}"."""),
            ],
        ),
    ]

    try:
        json_str = ""
        for chunk in client.models.generate_content_stream(
            model="gemini-2.0-flash",
            contents=contents,
            config=generate_content_config,
        ):
            json_str += chunk.text

        prompt = json.loads(json_str)["prompt"]

        logger.info(f"[SUCCESS] {word} ({word_type}):: {prompt}")

        return prompt

    except Exception as err:
        logger.error(f"[FUCKED] {word} ({word_type}):: {err!r}")
        return None

In [11]:
with open("./swedish_wordlist.csv", "r", encoding="UTF-8") as swedish_wordlist:
    reader = csv.DictReader(swedish_wordlist)
    wordlist = list(reader)

In [18]:
with open("./wordlist.csv", "r", encoding="UTF-8") as wordlist_f:
    reader = csv.DictReader(wordlist_f)
    wordlist_ids = [word_obj["id"] for word_obj in reader]

In [ ]:
for word_obj in wordlist:
    if word_obj["Index"] in wordlist_ids:
        continue
    
    time.sleep(5)
    
    prompt = generate_prompt(word_obj["English"], word_obj["Word Type"])
    if prompt is None:
        continue

    with open("./wordlist.csv", "a", encoding="UTF-8") as wordlist_f:
        wordlist_f.write(f"\"{word_obj['Index']}\",\"{word_obj['English']}\",\"{word_obj['Swedish']}\",\"{word_obj['Word Type']}\",\"{prompt}\"\n")